In [1]:
#RUN THIS
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#RUN THIS
!pip3 install transformers

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 860kB 67.6MB/s 
     |████████████████████████████████| 675kB 63.3MB/s 
     |████████████████████████████████| 1.0MB 48.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=e112226337153ed06a5974b4742a2c1a363dca01c6636716dd861147f131a698
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609179 sha256=4b4dc15b81186534051c0624a5b19ee10ba0de34414604fa35c81e788db2e3d8
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built sacremoses regex


In [3]:

import torch
import numpy as np
import pandas as pd
from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig

In [0]:


def read_data(filepath):
	print("reading ",filepath)
	genes = []
	labels = []
	df = pd.read_csv(filepath,usecols=[1,2],sep="\t",skiprows=1,header=None)
	print(df.head())
	for entry in df.itertuples():
		kmer_list = [kmer.strip("\'") for kmer in entry[1][1:-1].split(", ")]
		label_list = list(map(float, entry[2][1:-1].split(", ")))
		genes.append(kmer_list)
		labels.append(label_list)
	return genes, labels
	

def tokenize_samples(genes,labels):
  k= len(genes[0][0])
  if k==2:
    kmer_filepath = '/content/drive/My Drive/fourmersXL.txt'
  elif k==6:
    kmer_filepath = '/content/drive/My Drive/hexamersXL.txt'

  #formatted_samples = [['[CLS]']  + sample + ['[SEP]'] for sample in genes]
  #formatted_labels = [[0] + l + [0] for l in labels]
  tokenizer=TransfoXLTokenizer(vocab_file=kmer_filepath)
  print("TOKENIZER LENGTH", len(tokenizer))
  seq_ids = [tokenizer.convert_tokens_to_ids(gene) for gene in genes]
  
  return seq_ids, labels


WEIGHT = 20
def custom_MSE(preds,targets):
  weight_vec = torch.tensor([WEIGHT if t!=0 else 1 for t in targets]).long().cuda()
  first_pad = len(targets)
  #print(first_pad)
  #print(list(preds[:first_pad].detach().cpu().numpy().flatten()))
  #print((targets*15).detach().cpu().numpy().flatten())
  #print(list(preds-targets))
  #return torch.mean((WEIGHT*(preds[:first_pad] - targets))**2)/WEIGHT
  return torch.mean(((preds[:first_pad] - targets)**2)*weight_vec)

In [0]:
#RUN THIS
from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.config = TransfoXLConfig(vocab_size_or_config_json_file='/content/drive/My Drive/classification_XL_configuration.json')
    self.config.vocab_size=204098
    #self.config=TransfoXLConfig(vocab_size_or_config_json_file=204098,n_heads=8,n_layers=6,adaptive=False,tgt_len=1012,mem_len=1012)
    self.model = TransfoXLModel(self.config)
    self.out_layer = torch.nn.Linear(self.model.d_model, 1)
  def forward(self, input_ids, mems=None):
    hidden_states, mems = self.model(input_ids, mems)
    preds = self.out_layer(hidden_states[0]).squeeze(0).squeeze(1)
    return preds, mems


In [0]:
#RUN THIS
#setting some values
window_size = 1012
device = torch.device('cuda')
#building model
model = Model()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)



In [8]:
#RUN THIS
genes,labels = read_data('/content/drive/My Drive/Whole_Brain_val.txt')
gene_ids, labels = tokenize_samples(genes,labels)
print(gene_ids[0])
print(len(gene_ids))
print("Finished making data")

reading  /content/drive/My Drive/Whole_Brain_val.txt
                                                   1                                                  2
0  ['GGAGCC', 'CAGCCG', 'GGACTG', 'TCGCGC', 'GGGC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1  ['CTCAGC', 'TCTGCA', 'AAGAGC', 'TCCTCC', 'CCCT...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2  ['CAGGGA', 'CGTGTC', 'TGTGCT', 'CCTGCG', 'TGTG...  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
3  ['CGTGAG', 'GCCGGG', 'CAGGGG', 'GCGGGG', 'CCGG...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4  ['CTTCCG', 'CCAGCT', 'TCCCTC', 'CTCTTC', 'CTTT...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
TOKENIZER LENGTH 4098
[3895, 1241, 3869, 2527, 4057, 1889, 1839, 1911, 1409, 1535, 2959, 4087, 3576, 3901, 2935, 4053, 3582, 1214, 4032, 3870, 3198, 3960, 3385, 1692, 1809, 2869, 377, 3953, 2017, 1815, 2495, 3521, 87, 2013, 615, 1534, 449, 3480, 1904, 3751, 3431, 2495, 3059, 1432, 2367, 3027, 1239, 3397, 319, 2401, 3953, 

In [0]:
#RUN THIS

#how often to print stuff
genes_per_cycle=200
save_path = "/content/drive/My Drive/brain_6e_ft_6e.pt"

def save_weights():
  print("Saving weights")
  torch.save(model.state_dict(), save_path)


def run_epoch(genes,labels):
  epoch_loss=0
  cycle_loss = 0
  num_samples=len(genes)
  for i in range(num_samples):
    mems = None
    gene_loss = 0
    curr_gene = genes[i]
    curr_labels = labels[i]
    list_preds=[]
    #looping over all windows
    for w in range(0, len(curr_gene), window_size):
      toks = curr_gene[w:w+window_size]
      #ignore tiny windows(leads to dimensionality issues)
      if(len(toks)<2):
        continue
      window_input_ids = torch.tensor(toks).unsqueeze(0).cuda()
      window_labels = torch.tensor(curr_labels[w:w+window_size]).long().cuda()
      window_preds, mems = model(window_input_ids, mems)
      l = custom_MSE(window_preds, window_labels)
      gene_loss += l.item()
      #adds loss to storage
      l.backward()
      list_preds = list_preds+list(window_preds.detach().cpu().numpy())
    optimizer.step()
    optimizer.zero_grad()
    cycle_loss += gene_loss
    if i > 0 and i % genes_per_cycle == 0:
      np_preds=np.array(list_preds)
      print("Average Loss Per Gene: " + str(cycle_loss / genes_per_cycle))
      print("Max predicted value for one sample: ", np_preds.max())
      print("Average for same sample: ", np.average(np_preds))
      print("Top 10 Hits in Predictions",np.argsort(np_preds)[-10:])
      print("Top 10 Hits in Target",np.argsort(curr_labels)[-10:])
      epoch_loss+=cycle_loss
      cycle_loss = 0
    if i > 0 and i % 1000 == 0:
      save_weights()
  print("Loss on Epoch: "+str(epoch_loss/num_samples))
  save_weights()




In [10]:
#RUN THIS
f = "/content/drive/My Drive/brain_6e.pt"
model.load_state_dict(torch.load(f))
for i in range(6):
  run_epoch(gene_ids,labels)

Average Loss Per Gene: 0.04920856413191359
Max predicted value for one sample:  0.3884728
Average for same sample:  0.00062786107
Top 10 Hits in Predictions [6336 3537   12 2342 6682 4070 4601 1659   20 6233]
Top 10 Hits in Target [3987 6682 6934 3845 2697   20 3883 5777 5793 6632]
Average Loss Per Gene: 0.04654022412159975
Max predicted value for one sample:  0.34391633
Average for same sample:  0.023331206
Top 10 Hits in Predictions [ 656  158  111  623 1168  612  455  339  157  244]
Top 10 Hits in Target [326 339 300 312 366 348 656 798 558 577]
Average Loss Per Gene: 0.04291917342114175
Max predicted value for one sample:  0.3523344
Average for same sample:  0.0025283662
Top 10 Hits in Predictions [12418  4892  2349  8554  4238 12314  2095  9823  5476   229]
Top 10 Hits in Target [ 3680  5441  9021 10565 11741 11193 10732 11175 10712 10592]
Average Loss Per Gene: 0.04661791135493331
Max predicted value for one sample:  0.37577054
Average for same sample:  0.01422796
Top 10 Hits in 

KeyboardInterrupt: ignored

In [0]:
#Overfitting to small training sample
training_genes = genes[:10]
training_labels = labels[:10]
testing_genes=genes[10:11]
testing_labels=labels[10:11]
for i in range(0,30):
  print("Step ",i)
  model.train()
  run_epoch(training_genes,training_labels)
  eval_loss=evaluate(testing_genes,testing_labels)
  print(str(eval_loss))

